In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [2]:
class SpotifyAPI(object):
    access_token=None
    access_token_expiry = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded STRING
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception ("You must set client Id and client secret ")
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        
        return client_creds_b64.decode()
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}" 
}
    
    def get_token_data(self):
        return {
    "grant_type":"client_credentials"
}
    
    def perform_auth(self):
        
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url,data=token_data,headers=token_headers)
        
        if r.status_code not in range(200,299):
            return False
        
        data = r.json()
        now = datetime.datetime.now()
        self.access_token = data['access_token']
        expires_in = data['expires_in'] #in seconds
        expiry = now+datetime.timedelta(seconds=expires_in)
        self.access_token_expiry = expiry
        self.access_token_did_expire = expiry<now
        
        return True
        
        

In [3]:
import os

client_id = os.environ['SPOTIFY_CID']
client_secret = os.environ['SPOTIFY_CS']

In [4]:
spotify = SpotifyAPI(client_id,client_secret)

In [5]:
spotify.perform_auth()

True

In [6]:
access_token = spotify.access_token

In [7]:
headers = { 
    'Authorization':f'Bearer {access_token}'
}
endpoint = 'https://api.spotify.com/v1/search'
data = urlencode({"q":"Time","type":"track"})

lookup_url = f'{endpoint}?{data}'
print(lookup_url)
r=requests.get(lookup_url, headers=headers)
print(r.status_code)


https://api.spotify.com/v1/search?q=Time&type=track
200


In [9]:
data = urlencode({"q":"A lannister always pays his debts","type":"track"})

lookup_url = f'{endpoint}?{data}'
r=requests.get(lookup_url, headers=headers)